In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#importing necessary libraries
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
import numpy as np
from keras.preprocessing import image

In [ ]:
# Required additional libraries
import librosa
import numpy as np
import soundfile as sf
import os

# Audio Augmentation Functions
def add_noise(data, noise_level=0.005):
    noise = np.random.randn(len(data))
    return data + noise_level * noise

def time_stretch(data, rate=1.1):
    return librosa.effects.time_stretch(data, rate=rate)

def apply_pitch_shift(data, sr, n_steps=2):
    return librosa.effects.pitch_shift(y=data, sr=sr, n_steps=n_steps)

In [ ]:
def augment_and_save(audio_path, output_dir, file_name):
    y, sr = librosa.load(audio_path, sr=None)

    base_name = os.path.splitext(file_name)[0]  # safely remove extension

    # Save original again for consistency
    sf.write(os.path.join(output_dir, f"{base_name}_original.wav"), y, sr)

    # Add noise
    y_noise = add_noise(y)
    sf.write(os.path.join(output_dir, f"{base_name}_noise.wav"), y_noise, sr)

    # Time stretch (handle exception for very short clips)
    try:
        y_stretch = time_stretch(y, rate=1.2)
        sf.write(os.path.join(output_dir, f"{base_name}_stretch.wav"), y_stretch, sr)
    except:
        pass

    # Pitch shift
    y_pitch = apply_pitch_shift(y, sr, n_steps=2)
    sf.write(os.path.join(output_dir, f"{base_name}_pitch.wav"), y_pitch, sr)


In [ ]:
# Set the original and output directories
input_dir = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/recordings/"   # folder with original wav files
augmented_dir = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/augmented_audio/"    # folder to save augmented wav files

# Create output folder if it doesn't exist
os.makedirs(augmented_dir, exist_ok=True)

# Loop through and apply augmentations
for file in os.listdir(input_dir):
    if file.endswith(".wav"):
        audio_path = os.path.join(input_dir, file)
        augment_and_save(audio_path, augmented_dir, file)


In [ ]:
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav

def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
    # Read the audio file (returns sample rate and audio samples)
    sample_rate, samples = wav.read(audio_path)

    # If the audio is stereo (2 channels), convert it to mono by averaging the channels
    if samples.ndim > 1:
        samples = samples.mean(axis=1)

    # Create a new figure for the spectrogram
    fig = plt.figure()

    # Set figure size in inches based on desired spectrogram pixel dimensions and DPI
    fig.set_size_inches((spectrogram_dimensions[0] / fig.get_dpi(),
                         spectrogram_dimensions[1] / fig.get_dpi()))

    # Create an Axes object that fills the entire figure without margins
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()  # Turn off axis display (no ticks, labels, etc.)
    fig.add_axes(ax)   # Add the axes to the figure

    # Plot the spectrogram
    ax.specgram(samples, cmap=cmap, Fs=sample_rate, noverlap=noverlap)

    # Remove axis ticks
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    # Save the figure as an image, tightly cropped with no padding
    fig.savefig(save_path, bbox_inches="tight", pad_inches=0)


In [ ]:
from os import listdir
from os.path import isfile, join

def dir_to_spectrogram(audio_dir, spectrogram_dir, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
    """
    Converts all .wav files in a given directory to spectrogram images.

    Parameters:
    - audio_dir: Path to the folder containing .wav audio files.
    - spectrogram_dir: Path to the folder where spectrogram images will be saved.
    - spectrogram_dimensions: Size (width, height) of the output spectrogram image in pixels.
    - noverlap: Number of overlapping samples for the spectrogram calculation.
    - cmap: Color map for the spectrogram (default is grayscale reversed).
    """

    # Get list of all .wav files in the directory
    file_names = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f)) and '.wav' in f]

    # Loop over each .wav file
    for file_name in file_names:
        print(file_name)  # Optional: log the file being processed

        # Construct full path to the audio file
        audio_path = audio_dir + file_name

        # Construct path for saving the corresponding spectrogram image
        spectogram_path = spectrogram_dir + file_name.replace('.wav', '.png')

        # Call the function to convert audio to spectrogram image and save it
        wav_to_spectrogram(
            audio_path,
            spectogram_path,
            spectrogram_dimensions=spectrogram_dimensions,
            noverlap=noverlap,
            cmap=cmap
        )


In [ ]:
# Set the path to the folder containing audio (.wav) files
audio_folder = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/augmented_audio/"

# Set the path to the folder where spectrogram images will be saved
spectrogram_folder = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Spectrograms/"

# Call the function to convert all .wav files in the audio folder to spectrogram images
# Each spectrogram will be saved in the spectrogram_folder with the same base name and a .png extension
dir_to_spectrogram(audio_folder, spectrogram_folder)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_folder = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Spectrograms/"
train_folder = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Train/"
test_folder = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Test/"

# Create training and testing folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get list of spectrogram image files
files = [f for f in os.listdir(dataset_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
print("Total spectrogram files:", len(files))



In [ ]:
# Split the dataset into training and testing sets
train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)

# Move training files to the training folder
for file in train_files:
    shutil.move(os.path.join(dataset_folder, file), os.path.join(train_folder, file))

# Move testing files to the testing folder
for file in test_files:
    shutil.move(os.path.join(dataset_folder, file), os.path.join(test_folder, file))

In [ ]:
trainset = []  # Initialize an empty list to store image-label pairs
train_directory = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Train/"  # Path to the directory containing training images (e.g., spectrograms)

# Loop through each file in the training directory
for file in os.listdir(train_directory):

    # Extract the label from the file name
    labels = file.split('_')[0]

    # Extract the sample number (not used here but kept for reference)
    sample_number = file.split('_')[2]

    # Load the image using Keras (default target size can be added if needed)
    img = image.load_img(train_directory + file)

    # Convert the image to a NumPy array (for feeding into model)
    img_array = image.img_to_array(img)

    # Append the image array and its label as a pair to the dataset
    trainset.append([img_array, labels])


In [ ]:
#Similarly for test data
testset= []
test_directory = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Test/"
for file in os.listdir(test_directory):
  labels = file.split('_')[0]
  sample_number = file.split('_')[2]
  img = image.load_img(test_directory+file)
  testset.append([image.img_to_array(img), labels])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# Get only images in the train list, not the labels
X_train = [item[0] for item in trainset]
# Get only labels in the train list, not the images
y_train = [item[1] for item in trainset]
# Get only images in the test list, not the labels
X_test = [item[0] for item in testset]
# Get only labels in the test list, not the images
y_test = [item[1] for item in testset]


In [ ]:
# Convert list to numpy array in order to define input shape
X_train = np.asanyarray(X_train)
y_train = np.asanyarray(y_train)
X_test = np.asanyarray(X_test)
y_test = np.asanyarray(y_test)
print("X_train",X_train.shape)
print("y_train",y_train.shape)
print("X_test",X_test.shape)
print("y_test",y_test.shape)

In [ ]:
# Convert labels to numerical values
label_to_index = {label: i for i, label in enumerate(set(y_train))}
y_train_numeric = np.array([label_to_index[label] for label in y_train])
y_test_numeric = np.array([label_to_index[label] for label in y_test])

# Convert numerical labels to one-hot encoded vectors
num_classes = len(label_to_index)
y_train= to_categorical(y_train_numeric, num_classes=num_classes)
y_test= to_categorical(y_test_numeric, num_classes=num_classes)

# Normalize the images
X_train /= 255
X_test /= 255

In [ ]:
print(X_train.shape)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2] * X_train.shape[3]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2] * X_test.shape[3]))

print(X_train.shape)
print(X_test.shape)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(64, input_shape=(64, 192)))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))


In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf

from matplotlib.font_manager import FontProperties
from tensorflow import keras
font_prop = FontProperties(fname="/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Mangal Regular.ttf", size=12)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

confusion_mat = confusion_matrix(y_true_classes, y_pred_classes)

class_labels = ["शून्य", "एक", "दो", "तीन", "चार", "पाँच", "छह", "सात", "आठ", "नौ"]

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Set font properties for x-axis labels
plt.xticks(fontproperties=font_prop)

# Set font properties for y-axis labels
plt.yticks(fontproperties=font_prop)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

In [ ]:

history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Plot the accuracy versus epochs graph
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Evaluate model (optional)
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")

# Save the model
model.save("best_model.h5")
print("Model saved successfully as best_model.h5")

In [ ]:
import os
from os import listdir
from os.path import isfile, join

In [ ]:
folder_path= "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Test/"
def count_files_by_name(folder_path):
    file_count = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_name = os.path.splitext(file)[1]
            if file_name in file_count:
                file_count[file_name] += 1
            else:
                file_count[file_name] = 1
    return file_count

folder_path = input("Enter folder path: ")
file_count = count_files_by_name(folder_path)
for name, count in file_count.items():
    print(f"Number of files named '{name}': {count}")

In [ ]:
import os

def count_files_with_prefix(folder_path, prefix):
    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.startswith(prefix):
                file_count += 1
    return file_count

folder_path = "/content/drive/MyDrive/Colab Notebooks/RNNDigitClassification/Test/"

# Hindi digit prefixes from 0 (शून्य) to 9 (नौ)
hindi_digits = [
    "शून्य_",   # 0
    "एक_",   # 1
    "दो_",    # 2
    "तीन_",   # 3
    "चार_",   # 4
    "पाँच_",   # 5
    "छह_",   # 6
    "सात_",   # 7
    "आठ_",   # 8
    "नौ_"     # 9
]

# Loop through each digit and count files
for digit_prefix in hindi_digits:
    count = count_files_with_prefix(folder_path, digit_prefix)
    print(f"Number of files with names starting with '{digit_prefix}': {count}")
